In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
user_data = pd.read_csv("my_data/pos_user_data.csv", header=0, index_col=None)
print(user_data)

         userid  feedid  date_  device  read_comment  comment  like  \
0             8   71474      1       1             0        0     1   
1             8   73916      1       1             0        0     0   
2             8   50282      1       1             0        0     0   
3             8   11391      1       1             0        0     1   
4             8   27349      1       1             0        0     0   
...         ...     ...    ...     ...           ...      ...   ...   
6677724  250236   72813     12       2             0        0     0   
6677725  250236   52019     12       2             0        0     0   
6677726  250236   51045     12       2             0        0     0   
6677727  250236   46881     12       2             0        0     0   
6677728  250236   53804     12       2             1        0     0   

         click_avatar    play  
0                   0     500  
1                   0     250  
2                   0     750  
3                  

In [ ]:
os.mkdir("my_data/data_14/")

In [3]:
recommend_col = ['userid', 'feedid', 'device']
target_col = ['read_comment', 'comment', 'like', 'click_avatar']
history_col = ['userid', 'feedid', 'read_comment', 'comment', 'like', 'click_avatar']

In [4]:
train_recommend = pd.DataFrame(columns=recommend_col)
train_target = pd.DataFrame(columns=target_col)
train_history = pd.DataFrame(columns=history_col)
test_recommend = pd.DataFrame(columns=recommend_col)
test_target = pd.DataFrame(columns=target_col)
test_history = pd.DataFrame(columns=history_col)

In [5]:
user_id = user_data['userid'].unique()

In [6]:
for u in user_id:
    temp = user_data[user_data['userid'] == u]
    date = temp['date_'].unique()
    if len(date) >= 3:
        train_recommend = train_recommend.append(temp[temp['date_'] == date[-1]][train_recommend.columns])
        test_recommend = test_recommend.append(temp[temp['date_'] == date[-1]][test_recommend.columns])
        train_target = train_target.append(temp[temp['date_'] == date[-1]][train_target.columns])
        test_target = test_target.append(temp[temp['date_'] == date[-1]][test_target.columns])
        train_history = train_history.append(temp[temp['date_'] != date[-1]][train_history.columns])
        test_history = test_history.append(temp[temp['date_'] != date[-1]][test_history.columns])
    elif len(date) == 2:
        train_recommend = train_recommend.append(temp[temp['date_'] == date[-1]][train_recommend.columns])
        train_target = train_target.append(temp[temp['date_'] == date[-1]][train_target.columns])
        train_history = train_history.append(temp[temp['date_'] != date[-2]][train_history.columns])
    else:
        if len(temp) < 2:
            continue
        else:
            train_recommend = train_recommend.append(temp[-1: ][recommend_col])
            train_target = train_target.append(temp[-1: ][target_col])
            train_history = train_history.append(temp[ :-1][history_col])

In [7]:
train_recommend.to_csv("my_data/train/train_recommend.csv")
train_history.to_csv("my_data/train/train_history.csv")
train_target.to_csv("my_data/train/train_target.csv")
test_recommend.to_csv("my_data/test/test_recommend.csv")
test_history.to_csv("my_data/test/test_history.csv")
test_target.to_csv("my_data/test/test_target.csv")

In [10]:
from tensorflow.keras.layers.experimental import preprocessing
import pandas as pd
from tensorflow.keras import utils

my_info = pd.read_csv("my_data/my_info.csv", header=0, index_col=None)
my_info['manual_tag_list'] = my_info['manual_tag_list'].fillna(' ')
tag_data = my_info['manual_tag_list'].values
tag_token = preprocessing.TextVectorization(output_mode='binary')
tag_token.adapt(tag_data)
tag_vocabulary = tag_token.get_vocabulary()
print(len(tag_vocabulary))

351


In [ ]:

import numpy as np
my_info['manual_tag_list'] = my_info['manual_tag_list'].apply(lambda x: np.array(tag_token(x)).sum(axis=0))

In [ ]:
my_info.to_csv("my_data/my_info_gene.csv", index=None)

In [5]:
import pandas as pd
history_data = pd.read_csv("my_data/train_14/train_history.csv", header=0, index_col=0)

i_history = pd.DataFrame(columns=history_data.columns)
dis_history = pd.DataFrame(columns=['userid', 'feedid'])

users = history_data['userid'].unique()
history_data = history_data.groupby(by=['userid'])
for u in users:
    temp = history_data.get_group(u)
    i_history = i_history.append(temp[(temp['like'] == 1) | (temp['comment'] == 1) | (temp['read_comment'] == 1) | (temp['click_avatar'] == 1)])
    dis_history = dis_history.append(temp[(temp['like'] == 0) & (temp['comment'] == 0) & (temp['read_comment'] == 0) & (temp['click_avatar'] == 0)][['userid', 'feedid']])

E:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [1]:
import numpy as np
import pandas as pd
gene = pd.read_csv("my_data/train_14/my_info_gene.csv", header=0, index_col=None)

In [33]:
def convert_emb(x):
    return np.array(x.replace('[', '').replace(']', '').split(','), dtype=np.float)

def convert_tag(x):
    return np.array(x.replace('[', '').replace(']', '').replace('\n', '').split('. '), dtype=np.float)

emb = gene.values[:, -2]
tag = gene.values[:, -1]
emb = np.array(list(map(convert_emb, emb)))
tag = np.array(list(map(convert_tag, tag)))

In [34]:
gemb = np.average(emb, axis=0)
np.save('general_feed_emd.npy', np.average(emb, axis=0))

In [52]:
import pandas as pd
recom = pd.read_csv("my_data/train_14/train_recommend.csv", header=0, index_col=None)
target = pd.read_csv("my_data/train_14/train_target.csv", header=0, index_col=None)


all_recom = pd.concat([recom.reset_index(drop=True), target.reset_index(drop=True)], axis=1)


temp = pd.DataFrame(columns=all_recom.columns)
temp = temp.append(all_recom[(all_recom['like'] == 1) | (all_recom['forward'] == 1) | (all_recom['read_comment'] == 1) | (all_recom['click_avatar'] == 1)])

temp = temp.sample(frac=1)
temp_1 = temp[recom.columns]
tar_1= temp[target.columns]

temp_1.to_csv("my_data/train_14/train_recommend_true.csv", index=None)
tar_1.to_csv("my_data/train_14/train_target_true.csv", index=None)